Constructing Features from Scratch

In [8]:
#import libraries

import os
import sys
import numpy as np
import torchtext
import pandas as pd
import re
import requests
import nltk
from nltk.corpus import wordnet as wn
nltk.download('popular')

from lib import data_utils, preprocess

#get path
src_path = os.getcwd()

datapath = src_path + '/Data/'
PLcopath = datapath + 'AnnotatedPLData/PLCoref'
PLpath = datapath + 'AnnotatedPLData/PLTexts'
CENcopath = datapath + 'AnnotatedCENData/CENCoref'
CENpath = datapath + 'AnnotatedCENData/CENTexts'
ONpath = datapath + 'AnnotatedONData/ONCoref'

PLInter = src_path + '/output/IntermediateFilesPL/'
CENInter = src_path + '/output/IntermediateFilesCEN/'
ONInter = src_path + '/output/IntermediateFilesON/'

#import data

#create empty data frame
CEN_data = pd.DataFrame(columns=['corpusID', 'character', 'animacy', 'coref_chain', 'chain_head', 'head_of_head', 'chain_len', 'CL'])

#append all PL texts and features into one dataframe
for n in range(1,31):
# for n in range(1,2):

    print(n)

    #get story path
    storycopath = CENcopath +'/Novel' + str(n) + '.txt'
    storypath = CENpath + '/Novel' + str(n) + '.txt'
    storyid = 'Novel'+ str(n)

    #read in story
    corpus = data_utils.read_story(storycopath)

    # read in from intermediate files
    # list of features
    # features = ["CN", "Dep", "NER", "SS", "Triple", "WN"]

    # for f in features:
    #     #empty list
    #     feat = []
    #     with open(PLInter + f + 'FeatureBoolean'+'/Story' + str(n) + '.txt', 'r') as doc:
    #         for line in doc:
    #             feat.append(eval(line.rstrip()))
    
    #     corpus[f] = feat

    #get ss feature
    sslist = preprocess.semantic_subj(storypath)
    #remove leading The/A's in the sematic list
    sslist = [re.sub('^(The |A )','',s, flags=re.IGNORECASE) for s in sslist]
    pattern = '|'.join(sslist)
    pattern = pattern.replace('?|','')
    pattern = pattern.replace('!|','')
    pattern = pattern.replace('.|','')
    pattern = pattern.replace('(|','')
    pattern = pattern.replace('(','')
    pattern = pattern.replace(')','')
    
    #create binary flag variable for ss feat
    corpus['SS'] = corpus['head_of_head'].str.contains(pattern)
    corpus['SS'] = corpus['SS'].replace({True:1, False:0})

    #get ner feature
    nerlist = preprocess.ner_person(storypath)
    pattern = '|'.join(nerlist)
    pattern = pattern.replace('(','')
    pattern = pattern.replace(')','')
    pattern = pattern.replace('[','')
    pattern = pattern.replace(']','')

    #create binary flag variable for ss feat
    corpus['NER'] = corpus['head_of_head'].str.contains(pattern)
    corpus['NER'] = corpus['NER'].replace({True:1, False:0})


    #create binary flag variable for wn feat
    #get wordnet synset of head of chain
    wn_input = corpus['head_of_head'].apply(lambda word: pd.Series(wn.synsets(word)))
   
    #fill blanks with unrelated word to person
    wn_input[0] = wn_input[0].fillna(wn.synset('strong.a.01'))
   
    # get common synonym with person
    per = wn.synset('person.n.01')
    test = wn_input[0].apply(lambda syn: pd.Series(syn.lowest_common_hypernyms(per)))
   
    # test if head of chain related to person
    corpus['WN']= test[0]==per
    corpus['WN'] = corpus['WN'].replace({True:1, False:0})

    #get dp feat
    dplist = preprocess.dep_link(storypath)
    dplist = list(set(dplist))
    pattern='|'.join(dplist)
    #remove punct
    pattern = pattern.replace('?|','')
    pattern = pattern.replace('!|','')
    pattern = pattern.replace('.|','')
    pattern = pattern.replace('(|','')
    pattern = pattern.replace(')|','')
    pattern = pattern.replace('(','')
    pattern = pattern.replace(')','')

    #create binary flag variable for dp feat
    corpus['DP'] = corpus['head_of_head'].str.contains(pattern)
    corpus['DP'] = corpus['DP'].replace({True:1, False:0})

    #get triple feat
    tplist = preprocess.triple(storypath)
    tplist = list(set(tplist))
    #remove leading The/A's in the sematic list
    tplist = [re.sub('^(The |A )','',s, flags=re.IGNORECASE) for s in tplist]
    pattern='|'.join(tplist)
    pattern = pattern.replace('?|','')
    pattern = pattern.replace('!|','')
    pattern = pattern.replace('.|','')
    pattern = pattern.replace('(|','')
    pattern = pattern.replace(')|','')
    pattern = pattern.replace('(','')
    pattern = pattern.replace(')','')

    #create binary flag variable for ss feat
    corpus['TP'] = corpus['head_of_head'].str.contains(pattern)
    corpus['TP'] = corpus['TP'].replace({True:1, False:0})

    # get conceptnet feat
    urlreq = 'https://api.conceptnet.io/c/en/'+corpus['head_of_head']

    #default no presence of person mentioned
    corpus['CN'] = 0

    for i in range(len(urlreq)):

        #make request to concept net api
        response = requests.get(urlreq[i])
        obj = response.json()
        #get list of edges
        cnlist = [edge['@id'] for edge in obj['edges']]

        #if person is in list then flag
        if any('person' in s for s in cnlist):
            val = 1
            corpus['CN'][i]=val
    
    #create feature for term freq
    tf_dict = preprocess.term_freq(storypath)
    corpus['TF'] = corpus['head_of_head'].map(tf_dict)

    #append to dataframe
    CEN_data = pd.concat([CEN_data, corpus], ignore_index=True)

CEN_data

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/eileen/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/eileen/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/eileen/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/eileen/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/eileen/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/eileen/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to

1


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/tmp/ipykernel_291014/1036362030.py:93: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specif

2


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/tmp/ipykernel_291014/1036362030.py:93: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specif

...

29


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/tmp/ipykernel_291014/1036362030.py:93: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specif

30


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/tmp/ipykernel_291014/1036362030.py:93: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specif

,corpusID,character,animacy,coref_chain,chain_head,head_of_head,chain_len,CL,SS,NER,WN,DP,TP,CN,TF
0,Novel1,1,1,"[ Barold , He , him , Barold 's , Barold ,...",Barold,Barold,28,2.837451,0.0,1.0,0.0,1.0,1.0,0.0,0.003622
1,Novel1,1,1,"[ Lucia , Lucia , Lucia , Lucia , who had b...",Lucia,Lucia,17,1.487132,1.0,1.0,0.0,1.0,1.0,0.0,0.004074
2,Novel1,1,1,"[ Octavia , Octavia , I , Octavia , I , M...",Octavia,Octavia,24,2.346426,1.0,1.0,0.0,1.0,1.0,0.0,0.006338
3,Novel1,1,1,"[ Lady Theobald 's , furious Lady Theobald , ...",Lady Theobald 's,'s,12,0.873350,0.0,0.0,0.0,0.0,0.0,0.0,0.008601
4,Novel1,1,1,"[ Octavia and Lord Lansdowne , Lord Lansdowne...",Octavia and Lord Lansdowne,Lansdowne,16,1.364375,0.0,1.0,0.0,1.0,0.0,0.0,0.002716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5803,Novel30,0,0,[ them ],them,them,1,-0.218213,1.0,0.0,0.0,1.0,1.0,0.0,0.000000
5804,Novel30,0,0,[ Giovanni 's shoulders ],Giovanni 's shoulders,shoulders,1,-0.218213,0.0,0.0,0.0,0.0,0.0,0.0,0.000409
5805,Novel30,0,0,[ his angry eyes ],his angry eyes,eyes,1,-0.218213,1.0,0.0,0.0,1.0,0.0,0.0,0.002456
5806,Novel30,0,1,"[ My beloved , I , I , I , Especially I , ...",My beloved,beloved,10,0.812119,0.0,0.0,1.0,0.0,0.0,1.0,0.000409


Importing Features from Intermediate File

In [1]:
#import libraries

import os
import sys
import numpy as np
import torchtext
import pandas as pd
import re

from lib import data_utils, preprocess

#get path
src_path = os.getcwd()

datapath = src_path + '/Data/'
PLcopath = datapath + 'AnnotatedPLData/PLCoref'
PLpath = datapath + 'AnnotatedPLData/PLTexts'
CENcopath = datapath + 'AnnotatedCENData/CENCoref'
CENpath = datapath + 'AnnotatedCENData/CENCoref'
ONpath = datapath + 'AnnotatedONData/ONCoref'

PLInter = src_path + '/output/IntermediateFilesPL/'
CENInter = src_path + '/output/IntermediateFilesCEN/'
ONInter = src_path + '/output/IntermediateFilesON/'

#import data

#create empty data frame
CEN_Int_data = pd.DataFrame(columns=['corpusID', 'character', 'animacy', 'coref_chain', 'chain_head', 'head_of_head', 'chain_len', 'CL'])

#append all PL texts and features into one dataframe
for n in range(1,31):
# for n in range(1,2):

    print(n)

    #get story path
    storycopath = CENcopath +'/Novel' + str(n) + '.txt'
    storypath = CENpath + '/Novel' + str(n) + '.txt'
    storyid = 'Novel'+ str(n)

    #read in story
    corpus = data_utils.read_story(storycopath)

    #read in from intermediate files
    #list of features
    features = ["CN", "Dep", "NER", "SS", "Triple", "WN"]

    for f in features:
        #empty list
        feat = []
        with open(CENInter + f + 'FeatureBoolean'+'/Novel' + str(n) + '.txt', 'r') as doc:
            for line in doc:
                feat.append(eval(line.rstrip()))
    
        corpus[f] = feat
    
    #create feature for term freq
    tf_dict = preprocess.term_freq(storypath)
    corpus['TF'] = corpus['head_of_head'].map(tf_dict)
    
    #append to dataframe
    CEN_Int_data = pd.concat([CEN_Int_data, corpus], ignore_index=True)
CEN_Int_data

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


,corpusID,character,animacy,coref_chain,chain_head,head_of_head,chain_len,CL,CN,Dep,NER,SS,Triple,WN,TF
0,Novel1,1,1,"[ Barold , He , him , Barold 's , Barold ,...",Barold,Barold,28,2.837451,0.0,1.0,1.0,1.0,1.0,0.0,0.006784
1,Novel1,1,1,"[ Lucia , Lucia , Lucia , Lucia , who had b...",Lucia,Lucia,17,1.487132,0.0,1.0,1.0,1.0,0.0,0.0,0.008141
2,Novel1,1,1,"[ Octavia , Octavia , I , Octavia , I , M...",Octavia,Octavia,24,2.346426,0.0,1.0,0.0,1.0,0.0,0.0,0.013569
3,Novel1,1,1,"[ Lady Theobald 's , furious Lady Theobald , ...",Lady Theobald 's,'s,12,0.873350,0.0,0.0,1.0,0.0,1.0,0.0,0.006106
4,Novel1,1,1,"[ Octavia and Lord Lansdowne , Lord Lansdowne...",Octavia and Lord Lansdowne,Lansdowne,16,1.364375,0.0,1.0,1.0,1.0,1.0,0.0,0.003392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5803,Novel30,0,0,[ them ],them,them,1,-0.218213,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
5804,Novel30,0,0,[ Giovanni 's shoulders ],Giovanni 's shoulders,shoulders,1,-0.218213,0.0,0.0,1.0,0.0,0.0,0.0,0.000502
5805,Novel30,0,0,[ his angry eyes ],his angry eyes,eyes,1,-0.218213,0.0,1.0,0.0,1.0,0.0,0.0,0.002510
5806,Novel30,0,1,"[ My beloved , I , I , I , Especially I , ...",My beloved,beloved,10,0.812119,1.0,0.0,0.0,0.0,0.0,1.0,0.000502


In [9]:
CEN_data.to_csv("Data/CEN.csv")
CEN_Int_data.to_csv("Data/CEN_Inter.csv")

Model

In [12]:
# simple model with their features

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle
from joblib import dump, load

data_y = CEN_Int_data["character"].astype('int')
data_x = CEN_Int_data[["CL", "CN", "Dep", "NER", "SS", "Triple", "WN", "animacy"]]

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, random_state=0)

rbf_svc = svm.SVC(kernel = 'rbf', C=.5, gamma=1)

#fit
rbf_svc.fit(X_train, y_train)

#save
dump(rbf_svc, "models/CENjahan_model.joblib")
rbf_svc = load('models/CENjahan_model.joblib') 

cv_results = cross_validate(rbf_svc, X_train, y_train, cv = 10, scoring=('f1', 'accuracy', 'f1_macro'), return_train_score=True)

# print(cv_results['train_accuracy'])
# print(cv_results['test_accuracy'])
# print(cv_results['train_f1'])
# print(cv_results['test_f1'])

print("%0.2f accuracy with a standard deviation of %0.2f" % (cv_results['test_accuracy'].mean(), cv_results['test_accuracy'].std()))
print("%0.2f f1 score for character class with a standard deviation of %0.2f" % (cv_results['test_f1'].mean(), cv_results['test_f1'].std()))
print("%0.2f f1 score with a standard deviation of %0.2f" % (cv_results['test_f1_macro'].mean(), cv_results['test_f1'].std()))
print("%0.2f f1 score for non character class" % (2*cv_results['test_f1_macro'].mean() - cv_results['test_f1'].mean()))


0.98 accuracy with a standard deviation of 0.01
0.79 f1 score for character class with a standard deviation of 0.06
0.89 f1 score with a standard deviation of 0.06
0.99 f1 score for non character class


In [11]:
# simple model with their features

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle
from joblib import dump, load

data_y = CEN_Int_data["character"].astype('int')
data_x = CEN_Int_data[["CL", "CN", "Dep", "NER", "SS", "Triple", "WN", "animacy", "TF"]]

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, random_state=0)

rbf_svc_tf = svm.SVC(kernel = 'rbf', C=.5, gamma=1)

#fit
rbf_svc_tf.fit(X_train, y_train)

#save
dump(rbf_svc_tf, "models/CENjahan_model_tf.joblib")
rbf_svc_tf = load('models/CENjahan_model_tf.joblib') 

cv_results = cross_validate(rbf_svc_tf, X_train, y_train, cv = 10, scoring=('f1', 'accuracy', 'f1_macro'), return_train_score=True)

# print(cv_results['train_accuracy'])
# print(cv_results['test_accuracy'])
# print(cv_results['train_f1'])
# print(cv_results['test_f1'])

print("%0.2f accuracy with a standard deviation of %0.2f" % (cv_results['test_accuracy'].mean(), cv_results['test_accuracy'].std()))
print("%0.2f f1 score for character class with a standard deviation of %0.2f" % (cv_results['test_f1'].mean(), cv_results['test_f1'].std()))
print("%0.2f f1 score with a standard deviation of %0.2f" % (cv_results['test_f1_macro'].mean(), cv_results['test_f1'].std()))
print("%0.2f f1 score for non character class" % (2*cv_results['test_f1_macro'].mean() - cv_results['test_f1'].mean()))

0.98 accuracy with a standard deviation of 0.01
0.79 f1 score for character class with a standard deviation of 0.06
0.89 f1 score with a standard deviation of 0.06
0.99 f1 score for non character class


In [6]:
# simple model 

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle
from joblib import dump, load

CEN_data = pd.read_csv('Data/CEN.csv')

data_y = CEN_data["character"].astype('int')
data_x = CEN_data[["CL", "CN", "DP", "NER", "SS", "TP", "WN", "animacy"]]

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, random_state=1)

rbf_svc_1 = svm.SVC(kernel = 'rbf', C=.5, gamma=1)

#fit
rbf_svc_1.fit(X_train, y_train)

#save
dump(rbf_svc_1, "models/CENallen_model.joblib")
rbf_svc_1 = load('models/CENallen_model.joblib') 

cv_results = cross_validate(rbf_svc_1, X_train, y_train, cv = 10, scoring=('f1', 'accuracy', 'f1_macro'), return_train_score=True)

# print(cv_results['train_accuracy'])
# print(cv_results['test_accuracy'])
# print(cv_results['train_f1'])
# print(cv_results['test_f1'])
# print(cv_results['test_f1_macro'])

print("%0.2f accuracy with a standard deviation of %0.2f" % (cv_results['test_accuracy'].mean(), cv_results['test_accuracy'].std()))
print("%0.2f f1 score for character class with a standard deviation of %0.2f" % (cv_results['test_f1'].mean(), cv_results['test_f1'].std()))
print("%0.2f f1 score with a standard deviation of %0.2f" % (cv_results['test_f1_macro'].mean(), cv_results['test_f1'].std()))
print("%0.2f f1 score fro non character class" % (2*cv_results['test_f1_macro'].mean() - cv_results['test_f1'].mean()))

0.97 accuracy with a standard deviation of 0.01
0.74 f1 score for character class with a standard deviation of 0.05
0.86 f1 score with a standard deviation of 0.05
0.98 f1 score fro non character class


In [10]:
# simple model 

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle
from joblib import dump, load

CEN_data = pd.read_csv('Data/CEN.csv')

data_y = CEN_data["character"].astype('int')
data_x = CEN_data[["CL", "CN", "DP", "NER", "SS", "TP", "WN", "animacy", "TF"]]

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, random_state=1)

rbf_svc_1 = svm.SVC(kernel = 'rbf', C=.5, gamma=1)

#fit
rbf_svc_1.fit(X_train, y_train)

#save
dump(rbf_svc_1, "models/CENallen_model_tf.joblib")
rbf_svc_1 = load('models/CENallen_model_tf.joblib') 

cv_results = cross_validate(rbf_svc_1, X_train, y_train, cv = 10, scoring=('f1', 'accuracy', 'f1_macro'), return_train_score=True)

# print(cv_results['train_accuracy'])
# print(cv_results['test_accuracy'])
# print(cv_results['train_f1'])
# print(cv_results['test_f1'])
# print(cv_results['test_f1_macro'])

print("%0.2f accuracy with a standard deviation of %0.2f" % (cv_results['test_accuracy'].mean(), cv_results['test_accuracy'].std()))
print("%0.2f f1 score for character class with a standard deviation of %0.2f" % (cv_results['test_f1'].mean(), cv_results['test_f1'].std()))
print("%0.2f f1 score with a standard deviation of %0.2f" % (cv_results['test_f1_macro'].mean(), cv_results['test_f1'].std()))
print("%0.2f f1 score for non character class" % (2*cv_results['test_f1_macro'].mean() - cv_results['test_f1'].mean()))

0.97 accuracy with a standard deviation of 0.01
0.74 f1 score for character class with a standard deviation of 0.05
0.86 f1 score with a standard deviation of 0.05
0.98 f1 score fro non character class
